In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# About Dataset
#### The dataset contains Sales data of an Automobile company.

## Data Description

|Column Name|Description|
|----|----|
|ORDERNUMBER|This column represents the unique identification number assigned to each order.|
|QUANTITYORDERED|It indicates the number of items ordered in each order.|
|PRICEEACH|This column specifies the price of each item in the order.|
|ORDERLINENUMBER|It represents the line number of each item within an order.|
|SALES|This column denotes the total sales amount for each order, which is calculated by multiplying the quantity ordered by the price of each item.|
|ORDERDATE|It denotes the date on which the order was placed.|
|DAYS_SINCE_LASTORDER|This column represents the number of days that have passed since the last order for each customer. It can be used to analyze customer purchasing patterns.|
|STATUS|It indicates the status of the order, such as "Shipped," "In Process," "Cancelled," "Disputed," "On Hold," or "Resolved."|
|PRODUCTLINE|This column specifies the product line categories to which each item belongs.|
|MSRP|It stands for Manufacturer's Suggested Retail Price and represents the suggested selling price for each item.|
|PRODUCTCODE|This column represents the unique code assigned to each product.|
|CUSTOMERNAME|It denotes the name of the customer who placed the order.|
|PHONE|This column contains the contact phone number for the customer.|
|ADDRESSLINE1|It represents the first line of the customer's address.|
|CITY|This column specifies the city where the customer is located.|
|POSTALCODE|It denotes the postal code or ZIP code associated with the customer's address.|
|COUNTRY|This column indicates the country where the customer is located.|
|CONTACTLASTNAME|It represents the last name of the contact person associated with the customer.|
|CONTACTFIRSTNAME|This column denotes the first name of the contact person associated with the customer.|
|DEALSIZE|It indicates the size of the deal or order, which are the categories "Small," "Medium," or "Large."|

In [ ]:
# other libraries required for EDA

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

sns.set() # setting seaborn as default style

In [ ]:
# read the dataset in pandas Dataframe

# dateparser
dateparser = lambda x : datetime.strptime(x, "%d/%m/%Y")

# read csv in pandas Dataframe. Change the datatype to datetime and column name of order date (ORDERDATE) column
df_autosales = pd.read_csv("/kaggle/input/auto-sales-data/Auto Sales data.csv", \
                           parse_dates = {"order_date" : ["ORDERDATE"]}, date_parser = dateparser)

In [ ]:
# glance of the dataset
df_autosales.head()

In [ ]:
# list of columns name
df_autosales.columns.tolist()

In [ ]:
# creating a new column order year and converting the values of order date column to "monthname year"
df_autosales["order_year"] = df_autosales["order_date"].dt.year
df_autosales["order_date"] = df_autosales["order_date"].dt.strftime("%b %Y")
df_autosales.head()

In [ ]:
# drop columns which are not required
df_autosales.drop(columns=["ORDERNUMBER", "PRICEEACH", "ORDERLINENUMBER", "MSRP", \
                           "PRODUCTCODE", "PHONE", "ADDRESSLINE1", "CITY", "POSTALCODE", \
                           "DAYS_SINCE_LASTORDER", "CONTACTLASTNAME", "CONTACTFIRSTNAME"], inplace=True)

In [ ]:
# creating a new column from the values of customer name column and country
df_autosales["customer"] = df_autosales["CUSTOMERNAME"] + " (" + df_autosales["COUNTRY"] + ")"
df_autosales.drop(columns=["CUSTOMERNAME", "COUNTRY"], inplace = True)

In [ ]:
# rename columns
df_autosales.rename(columns={"QUANTITYORDERED": "order_quantity", "SALES": "sales_amount", \
                             "STATUS": "order_status", "PRODUCTLINE": "productline", \
                             "DEALSIZE": "dealsize"}, inplace = True)

In [ ]:
# find total null values in each column
df_autosales.isnull().sum()

In [ ]:
# find duplicate records
df_autosales.duplicated().sum()

In [ ]:
# rearrange the columns
df_autosales = df_autosales[["customer", "order_date", "order_year", "order_quantity", "productline", \
                 "sales_amount", "dealsize", "order_status"]]

In [ ]:
# final dataframe details
df_autosales.info()

##### Breakup of final dataframe details

In [ ]:
# final dataframe shape
df_autosales.shape

In [ ]:
# final dataframe datatype
df_autosales.dtypes

In [ ]:
# statistical view of the final dataframe
df_autosales.describe(include = "all")

# Descriptive Analytics
## Plots for each customers in the dataset:
* date wise productline orders
* year wise sales amount
* year wise sales amount percentage
* date wise productline orders dealsize count
* date wise productline orders status count

In [ ]:
customers = pd.unique(df_autosales["customer"]).tolist()

for customer in customers:
    print("\033[1m",customer,"\033[0m.")
    customer_df = df_autosales[df_autosales["customer"] == customer]
    plt.figure(figsize=(21, 15))
    datewise_orders = customer_df.groupby(["order_date", "productline"], as_index = False)["order_quantity"].sum()
    datewise_orders["productordered_datewise"] = datewise_orders["productline"] + " (" + datewise_orders["order_date"] + ")"
    datewise_orders.drop(columns=["order_date", "productline"], inplace = True)
    datewise_orders = datewise_orders[["productordered_datewise", "order_quantity"]]
    sns.barplot(x=datewise_orders["productordered_datewise"], y=datewise_orders["order_quantity"], data=datewise_orders)
    plt.title( customer + " date wise productline orders", fontsize=21)
    plt.xticks(rotation=90, fontsize=16)
    plt.xlabel("productordered_datewise", fontsize=16)
    plt.show()
    print("\n")
    print("\n")
    yearwise_salesamt = customer_df.groupby(["order_year"], as_index = False)["sales_amount"].sum()
    sns.barplot(x=yearwise_salesamt["order_year"], y=yearwise_salesamt["sales_amount"], data=yearwise_salesamt)
    plt.title( customer + " year wise sales amount")
    plt.xticks(rotation=90)
    plt.show()
    print("\n")
    print("\n")
    colors = sns.color_palette('pastel')
    plt.pie(yearwise_salesamt["sales_amount"], labels = yearwise_salesamt["order_year"], colors = colors, autopct='%.0f%%')
    plt.title( customer + " year wise sales amount percentage")
    plt.xticks(rotation=90)
    plt.show()
    print("\n")
    print("\n")
    datewiseorders_dealsize = customer_df.groupby(["order_date", "productline"], as_index = False)["dealsize"].value_counts()
    datewiseorders_dealsize["productordered_datewise_dealsize"] = datewiseorders_dealsize["productline"] + \
    " (" + datewiseorders_dealsize["order_date"] + ")" + " (" + datewiseorders_dealsize["dealsize"] + " dealsize)"
    datewiseorders_dealsize.drop(columns=["order_date", "productline", "dealsize"], inplace = True)
    datewiseorders_dealsize = datewiseorders_dealsize[["productordered_datewise_dealsize", "count"]]
    plt.figure(figsize=(21, 15))
    sns.barplot(x=datewiseorders_dealsize["productordered_datewise_dealsize"], y=datewiseorders_dealsize["count"], data=datewiseorders_dealsize)
    plt.title( customer + " date wise productline orders dealsize count", fontsize=21)
    plt.xticks(rotation=90, fontsize=16)
    plt.xlabel("productordered_datewise_dealsize", fontsize=16)
    plt.show()
    print("\n")
    print("\n")
    datewiseorders_status = customer_df.groupby(["order_date", "productline"], as_index = False)["order_status"].value_counts()
    datewiseorders_status["datewiseproductordered_status"] = datewiseorders_status["productline"] + \
    " (" + datewiseorders_status["order_date"] + ")" + " (order status: " + datewiseorders_status["order_status"] + ")"
    datewiseorders_status.drop(columns=["order_date", "productline", "order_status"], inplace = True)
    datewiseorders_status = datewiseorders_status[["datewiseproductordered_status", "count"]]
    plt.figure(figsize=(21, 15))
    sns.barplot(x=datewiseorders_status["datewiseproductordered_status"], y=datewiseorders_status["count"], data=datewiseorders_status)
    plt.title( customer + " date wise productline orders status count", fontsize=21)
    plt.xticks(rotation=90, fontsize=16)
    plt.xlabel("datewiseproductordered_status", fontsize=16)
    plt.show()
    print("\n")
    print("\n")
    print("\n")
    print("\n")
    print("\n")
    


# Descriptive Analytics
## Plots for each productline in the dataset:
* date wise orders
* year wise sales amount
* year wise sales amount percentage
* date wise productline orders dealsize count
* date wise productline orders status count

In [ ]:
products = pd.unique(df_autosales["productline"]).tolist()

for product in products:
    print("\033[1m",product,"\033[0m.")
    product_df = df_autosales[df_autosales["productline"] == product]
    plt.figure(figsize=(21, 15))
    datewise_orders = product_df.groupby(["order_date"], as_index = False)["order_quantity"].sum()
    datewise_orders = datewise_orders[["order_date", "order_quantity"]]
    sns.barplot(x=datewise_orders["order_date"], y=datewise_orders["order_quantity"], data=datewise_orders)
    plt.title( product + " date wise orders", fontsize=21)
    plt.xticks(rotation=90, fontsize=16)
    plt.xlabel("order_date", fontsize=16)
    plt.show()
    print("\n")
    print("\n")
    yearwise_salesamt = product_df.groupby(["order_year"], as_index = False)["sales_amount"].sum()
    sns.barplot(x=yearwise_salesamt["order_year"], y=yearwise_salesamt["sales_amount"], data=yearwise_salesamt)
    plt.title( product + " year wise sales amount")
    plt.xticks(rotation=90)
    plt.show()
    print("\n")
    print("\n")
    colors = sns.color_palette('pastel')
    plt.pie(yearwise_salesamt["sales_amount"], labels = yearwise_salesamt["order_year"], colors = colors, autopct='%.0f%%')
    plt.title( product + " year wise sales amount percentage")
    plt.xticks(rotation=90)
    plt.show()
    print("\n")
    print("\n")
    datewiseorders_dealsize = product_df.groupby(["order_date"], as_index = False)["dealsize"].value_counts()
    datewiseorders_dealsize = datewiseorders_dealsize[["order_date", "count"]]
    plt.figure(figsize=(21, 15))
    sns.barplot(x=datewiseorders_dealsize["order_date"], y=datewiseorders_dealsize["count"], data=datewiseorders_dealsize)
    plt.title( product + " date wise orders dealsize count", fontsize=21)
    plt.xticks(rotation=90, fontsize=16)
    plt.xlabel("order_date", fontsize=16)
    plt.show()
    print("\n")
    print("\n")
    datewiseorders_status = product_df.groupby(["order_date"], as_index = False)["order_status"].value_counts()
    datewiseorders_status = datewiseorders_status[["order_date", "count"]]
    plt.figure(figsize=(21, 15))
    sns.barplot(x=datewiseorders_status["order_date"], y=datewiseorders_status["count"], data=datewiseorders_status)
    plt.title( product + " date wise orders status count", fontsize=21)
    plt.xticks(rotation=90, fontsize=16)
    plt.xlabel("order_date", fontsize=16)
    plt.show()
    print("\n")
    print("\n")
    print("\n")
    print("\n")
    print("\n")

## End
### If you found this EDA to be interesting please do upvote my notebook
### Also please do comment with your valuable feedback